<a href="https://colab.research.google.com/github/yenee/class/blob/main/report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1.Import



In [ ]:
!pip install pingouin

In [3]:
import pingouin as pg
import pandas as pd
import io

In [57]:
import statsmodels.formula.api as smf


def forward_selected(data, response):
    """Linear model designed by forward selection.

    Parameters:
    -----------
    data : pandas DataFrame with all possible predictors and response

    response: string, name of response column in data

    Returns:
    --------
    model: an "optimal" fitted statsmodels linear model
           with an intercept
           selected by forward selection
           evaluated by adjusted R-squared
    """
    remaining = set(data.columns)
    remaining.remove(response)
    selected = []
    current_score, best_new_score = 0.0, 0.0
    while remaining and current_score == best_new_score:
        scores_with_candidates = []
        for candidate in remaining:
            formula = "{} ~ {} + 1".format(response,
                                           ' + '.join(selected + [candidate]))
            score = smf.ols(formula, data).fit().rsquared_adj
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates.pop()
        if current_score < best_new_score:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
    formula = "{} ~ {} + 1".format(response,
                                   ' + '.join(selected))
    model = smf.ols(formula, data).fit()
    return model

In [85]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [80]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

#2.Reading data


In [18]:
from google.colab import files
uploaded = files.upload()

Saving mediandatabird.csv to mediandatabird (1).csv


In [ ]:
df = pd.read_csv(io.BytesIO(uploaded['mediandatabird.csv']),encoding='GB18030')
df = df.drop(columns=['id','land','water','num','Pw','Pc'])
df.columns = ["outbreak","FF","MD","FW","FO","WL","UB","WB","BU","BL","長趾濱鷸","紅嘴鷗","小","田鷸","尖尾鴨","太平洋金斑","黑翅長腳鷸","小燕鷗","家燕","黃頭鷺","PHI"]

# Dataset is now stored in a Pandas Dataframe

#3.Regression Model


In [86]:
X = df[["FF","MD","FW","FO","WL","UB","WB","BU","BL","長趾濱鷸","紅嘴鷗","小","田鷸","尖尾鴨","太平洋金斑","黑翅長腳鷸","小燕鷗","家燕","黃頭鷺","PHI"]]
# VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]
print(vif_data)

   feature       VIF
0       FF  2.380984
1       MD  2.128165
2       FW  3.046372
3       FO  5.883675
4       WL  2.038363
5       UB  2.242118
6       WB  2.319907
7       BU  6.252079
8       BL  1.082843
9     長趾濱鷸  2.013180
10     紅嘴鷗  1.703656
11       小  1.975015
12      田鷸  1.860415
13     尖尾鴨  1.775713
14   太平洋金斑  2.082723
15   黑翅長腳鷸  2.942905
16     小燕鷗  2.073074
17      家燕  5.708494
18     黃頭鷺  6.130247
19     PHI  1.750764


In [64]:
model = forward_selected(df,'outbreak')
print(model.model.formula)

outbreak ~ PHI + BU + UB + FW + 黑翅長腳鷸 + FO + FF + 長趾濱鷸 + 1


In [65]:
lm = pg.linear_regression(df[['PHI','BU','UB','FW','黑翅長腳鷸','FO','FF','長趾濱鷸']], df['outbreak'],relimp=True)

#4.LMG

In [79]:
lmg = pd.DataFrame(lm[['names','relimp']])
lmg = lmg.drop(lmg.index[0])
lmg

,names,relimp
1,PHI,0.139093
2,BU,0.025656
3,UB,0.014469
4,FW,0.032627
5,黑翅長腳鷸,0.017033
6,FO,0.033282
7,FF,0.002200
8,長趾濱鷸,0.004656


#5.RF

In [112]:
X = df[['PHI','BU','UB','FW','黑翅長腳鷸','FO','FF','長趾濱鷸']]
y = df['outbreak'] 

rfc=RandomForestClassifier(n_estimators=5, oob_score = True)
rfc.fit(X,y)
print('Score: ', rfc.score(X, y))
print(rfc.oob_score_)

Score:  0.9161230195712954
0.7651444547996272


/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_forest.py:564: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  UserWarning,


In [93]:
rf0 = RandomForestClassifier(oob_score=True, random_state=10)
rf0.fit(X_train,y_train)
print(rf0.oob_score_)

0.7800559179869525


In [110]:
sample_n_estimator = [50,100,150,200,250,300,350,400,450,500]

for n_size in sample_n_estimator :
  model = RandomForestClassifier(n_estimator=n_size ,oob_score=True)
  model.fit(X_train,y_train)
  print(n_size,model.oob_score_)

TypeError: ignored

In [134]:
rf0 = RandomForestClassifier(oob_score=True, random_state=10)
rf0.fit(X,y)
rf0.oob_score_

0.7651444547996272


In [126]:
param_test1 = {'n_estimators':range(100,501,50)}
gsearch1 = GridSearchCV(estimator = RandomForestClassifier(min_samples_split=100,max_features='sqrt' ,random_state=10), 
                       param_grid = param_test1, scoring='roc_auc',cv=5)
gsearch1.fit(X,y)
gsearch1.best_params_, gsearch1.best_score_

({'n_estimators': 200}, 0.8330065469431244)

In [127]:
param_test3 = {'min_samples_split':range(80,150,20), 'min_samples_leaf':range(10,60,10)}
gsearch3 = GridSearchCV(estimator = RandomForestClassifier(n_estimators= 200,
                                max_features='sqrt' ,oob_score=True, random_state=10),
                                param_grid = param_test3, scoring='roc_auc',cv=5)
gsearch3.fit(X,y)
gsearch3.best_params_, gsearch3.best_score_

({'min_samples_leaf': 10, 'min_samples_split': 120}, 0.8340668433079588)

In [132]:
param_test4 = {'max_features':range(3,8,1)}
gsearch4 = GridSearchCV(estimator = RandomForestClassifier(n_estimators= 200,min_samples_split=120,
                              min_samples_leaf=10 ,oob_score=True, random_state=10),
                              param_grid = param_test4, scoring='roc_auc', cv=5)
gsearch4.fit(X,y)
gsearch4.best_params_, gsearch4.best_score_

({'max_features': 4}, 0.8330230194445611)

In [133]:
rf2 = RandomForestClassifier(n_estimators= 200,min_samples_split=120,
                min_samples_leaf=10, max_features=4 ,oob_score=True, random_state=10)
rf2.fit(X,y)
rf2.oob_score_

0.7921714818266542